## HW3. Stochastic Gradient Descent

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
classes = ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica')

In [44]:
data = pd.read_csv('iris.data', header=None)

In [63]:
data

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [45]:
from sklearn.preprocessing import StandardScaler

In [68]:
train, test = train_test_split(data, train_size=0.9, random_state=42, stratify=data[4])
train.shape, test.shape

((135, 5), (15, 5))

In [83]:
X_train = train[[0, 1, 2, 3]]
y_train = train[4]
X_test = test[[0, 1, 2, 3]]
y_test = test[4]

In [84]:
scaler = StandardScaler()
scaler.fit(X_train)
train_scaled = pd.DataFrame(scaler.transform(X_train))
train_scaled[4] = y_train.reset_index(drop=True)
test_scaled = pd.DataFrame(scaler.transform(X_test))
test_scaled[4] = y_test.reset_index(drop=True)

In [85]:
test_scaled

,0,1,2,3,4
0,0.937066,-0.348086,0.484926,0.124931,Iris-versicolor
1,0.328874,-1.029122,1.056673,0.255674,Iris-virginica
2,0.815428,-0.121073,0.827974,1.040127,Iris-virginica
3,-0.279318,-1.256135,0.084703,-0.136553,Iris-versicolor
4,1.788535,-0.348086,1.456896,0.778642,Iris-virginica
5,-1.738979,0.332951,-1.401839,-1.313232,Iris-setosa
6,-1.738979,-0.121073,-1.401839,-1.313232,Iris-setosa
7,-1.009148,0.786976,-1.287490,-1.313232,Iris-setosa
8,0.693789,-0.575098,1.056673,1.301611,Iris-virginica
9,0.937066,-0.121073,0.370577,0.255674,Iris-versicolor


In [88]:
train_scaled.to_csv('train.csv', header=False, index=False)
test_scaled.to_csv('test.csv', header=False, index=False)

X_train = train_scaled[[0, 1, 2, 3]]
y_train = train_scaled[4]
X_test = test_scaled[[0, 1, 2, 3]]
y_test = test_scaled[4]

### Task 1.

In [50]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [52]:
def compute_gradient(x, y, w):
    grad = np.dot(x.T, sigmoid(x.dot(w)) - y)
    return grad if len(x.shape) == 1 else grad / len(x)